<a href="https://colab.research.google.com/github/shivashankar066/AI_Resolution_System/blob/main/Fine_Tuning_LLm_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.1/171.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 6.8 MB/s eta 0:00:00


In [2]:
import os
os.environ['GRADIENT_WORKSPACE_ID']='e64e6c39-88f8-410b-b886-475418118177_workspace'
os.environ['GRADIENT_ACCESS_TOKEN']='SUfjRYkUIcWnKhd2AteCIozIvdf993nG'

In [3]:
from gradientai import Gradient


def main():
    gradient = Gradient()

    base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

    new_model_adapter = base_model.create_model_adapter(
        name="Shivashankar_model"
    )
    print(f"Created model adapter with id {new_model_adapter.id}")


    sample_query = "### Instruction: Who is shivashankar? \n\n ### Response:"
    print(f"Asking: {sample_query}")
    ## Before Finetuning
    completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
    print(f"Generated(before fine tuning): {completion}")

    samples=[
        {"inputs":"### Instruction: Who is shivashankar? \n\n### Response: shiva is a popular mentor and He has knowledge on Data Science,AI And LLM"}
        ]

    ## Lets define parameters for finetuning
    num_epochs=3
    count=0
    while count < num_epochs:
      print(f"Fine tuning the model with iteration {count + 1}")
      new_model_adapter.fine_tune(samples=samples)
      count=count+1

    #after fine tuning
    completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
    print(f"Generated(after fine tuning): {completion}")
    # new_model_adapter.delete()
    gradient.close()

if __name__ == "__main__":
    main()

Created model adapter with id ba2f7125-2309-467c-95ed-59392eb40f5c_model_adapter
Asking: ### Instruction: Who is shivashankar? 

 ### Response:
Generated(before fine tuning):  Shivashankar is an Indian athlete who specializes in the 110 meter hurdles. He is a national record holder and has won several medals at international competitions, including a bronze medal at the 2018 Asian Games.
Fine tuning the model with iteration 1
Fine tuning the model with iteration 2
Fine tuning the model with iteration 3
Generated(after fine tuning):  shiva is a popular mentor and He has knowledge on Data Science,AI And LLMs.


In [5]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00


In [6]:
from langchain.chains import LLMChain
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate

In [7]:
import os
from getpass import getpass

if not os.environ.get("GRADIENT_ACCESS_TOKEN", None):
    # Access token under https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_ACCESS_TOKEN"] = getpass("SUfjRYkUIcWnKhd2AteCIozIvdf993nG")


In [8]:
if not os.environ.get("GRADIENT_WORKSPACE_ID", None):
    # `ID` listed in `$ gradient workspace list`
    # also displayed after login at at https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_WORKSPACE_ID"] = getpass("e64e6c39-88f8-410b-b886-475418118177_workspace")

In [9]:
import gradientai

client = gradientai.Gradient()

models = client.list_models(only_base=True)
for model in models:
    print(model.id)

99148c6d-c2a0-4fbe-a4a7-e7c05bdb8a09_base_ml_model
f0b97d96-51a8-4040-8b22-7940ee1fa24e_base_ml_model
cc2dafce-9e6e-4a23-a918-cad6ba89e42e_base_ml_model


In [10]:
new_model = models[-1].create_model_adapter(name="my_model_adapter")
new_model.id, new_model.name

('1e86dd2e-7bec-41f1-afb1-9fda75a27cc1_model_adapter', 'my_model_adapter')

In [11]:
llm = GradientLLM(
    # `ID` listed in `$ gradient model list`
    model="1e86dd2e-7bec-41f1-afb1-9fda75a27cc1_model_adapter",
    # # optional: set new credentials, they default to environment variables
    # gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    # gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    model_kwargs=dict(max_generated_token_count=128),
)

In [15]:
template = """Question: {question}

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [16]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [17]:
question = "who is shivashankar?"

llm_chain.run(question=question)

'1. Who is Shivashankar?\nShivashankar is an Indian name that can be used for both males and females. It is derived from the Sanskrit words "shiva" meaning "auspicious" and "shankar" meaning "benefactor." The name is often associated with Lord Shiva, one of the main deities in Hinduism.\n2. What is Shivashankar known for?\nShivashankar is not a specific person, but rather a name that can be given to people. Therefore, there is no specific thing that Shiv'

In [18]:
dataset = [
    {
        "inputs": template.format(question="What NFL team won the Super Bowl in 1994?")
        + " The Dallas Cowboys!"
    }
]
dataset

[{'inputs': 'Question: What NFL team won the Super Bowl in 1994?\n\nAnswer:  The Dallas Cowboys!'}]

In [19]:
new_model.fine_tune(samples=dataset)

FineTuneResponse(number_of_trainable_tokens=27, sum_loss=76.17672)

In [20]:
# we can keep the llm_chain, as the registered model just got refreshed on the gradient.ai servers.
llm_chain.run(question=question)

' Shivashankar is a 13-year-old boy from Karnataka, India who has won the National Geographic Bee competition for the year 2015.'